In [61]:
import torch
import cv2
import numpy as np
from dataset import create_datasets
from model import Net
from anchor import generate_anchors, mark_anchors

## 创建Datasets && 随机看一些图片和真值

In [32]:
train_dataset, val_dataset = create_datasets('/home/louis/datasets/wider_face')
# TODO: show some images and ground truth bbox here

## 创建模型并随机用某张图片正向获得Feature Maps

In [47]:
image, annotation = train_dataset[0]
height, width = image.shape[:2]
image = cv2.resize(image, (640, 640))

height_scale = 640.0 / height
width_scale, height_scale = 640.0 / width, 640.0 / height

_input = torch.tensor(image).permute(2, 0, 1).unsqueeze(0).float()
net = Net()
feature_maps = list(net(_input))

In [34]:
for index, feature_map in enumerate(feature_maps):
    feature_maps[index] = feature_map.view(1, 6, -1) # 1 should be replaced by batch size

### 每个Feature Map的预测数量

In [35]:
feature_map_predictions = map(lambda x: x.size()[2], feature_maps)

print("predictions(4 + 2) of every feature map(from bottom to up) is {}".format( list(feature_map_predictions) ) )

predictions(4 + 2) of every feature map(from bottom to up) is [24964, 6084, 1444, 400, 100, 25]


## 生成的Anchor

In [36]:
anchors = generate_anchors() # 使用默认设置

In [37]:
anchors_number = map(lambda x: len(x), anchors )
print("anchors number for every scale (from bottom to up) is {}".format(  list(anchors_number) ) )

anchors number for every scale (from bottom to up) is [25600, 6400, 1600, 400, 100, 25]


### 使用Ground Truth来标记Anchor

In [58]:
gt_bboxes_scaled = list(map(lambda x: [  x[0] * height_scale, x[1] * width_scale, x[2] * height_scale, x[3] * width_scale ] , annotation))

## TODO: anchors 是否应该和 Feature Map 的 predictions 保持一致？？？？